In [16]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Para listar los archivos de una carpeta
import os
# Para importar archivos .mat
import scipy.io
# Para String to dateTime
import datetime
# 2d to 3d
from numpy import zeros, newaxis
# Para redimensionar
from PIL import Image
# Para abrir y guardar archivos csv
import csv

# Para imagenes binarias NO USADO TODAVIA
from scipy import ndimage

## Funciones Auxiliares

In [17]:
# Obtiene las plantillas dada la camara por parámetro
def getPlantillas(cam):
    carpeta = "./Plantillas/" + cam
    listPlantillas = os.listdir(carpeta)
    variable = sp.io.loadmat(carpeta + "/" + listPlantillas[0])
    variable2 = np.array(variable["I_Plantilla"])
    plantillas = variable2[:, :, newaxis]    
    for i in range(1,len(listPlantillas)):
        pltil = sp.io.loadmat(carpeta + "/" + listPlantillas[i])
        pltil2 = np.array(pltil["I_Plantilla"])
        pltil2 = pltil2[:, :, newaxis]
        plantillas = np.concatenate([plantillas, pltil2], axis = 2)
    return plantillas

In [18]:
# Visualiza plantillas pasadas por parámetro
def visualizaPlantillas(plantillas):
    tam = plantillas.shape
    a = np.array_split(plantillas,tam[2],axis=2)
    for i in range(0,len(a)):
        plt.figure()
        plt.imshow(a[i][:][:])

In [19]:
# True si la imagen pasada por parámetro es disponible
def camDisponible(iEstudio):
    disp = False
    noDisponible = Image.open('./No_Disponible/NO_DISPONIBLE.jpg')
    tam = noDisponible.size
    iEs = Image.fromarray(iEstudio)
    iEsRed = iEs.resize(tam)
    noArray = np.array(noDisponible)
    iEsArray = np.array(iEsRed)
    dif = iEsArray - noArray
    maximo = np.max(dif)
    if maximo == 0:
        disponible = False
    else:
        disponible = True
    return disponible

In [20]:
# Devuleve True si se puede calcular el fondo, en caso contrario, significa que tenemos una imagen NO DISPONIBLE
def isFondoDisponible(cam, listImagenes):
    disponible = True
    encontrado = False
    i = 0
    while i < (len(listImagenes) - 1) and encontrado == False:
        disp = camDisponible(plt.imread("./Datos/"+ cam + "/" + listImagenes[i]))
        if disp:
            i += 1
        else:
            encontrado = True
            disponible = False
    return disponible

In [21]:
def calculaFondo(cam, listImagenes):
    imgsFondo = plt.imread("./Datos/"+ cam + "/" + listImagenes[0])
    tam = imgsFondo.shape
    imgsFondo = imgsFondo[:, :, :, newaxis]    
    for i in range(1, len(listImagenes) - 1):
        imgF = plt.imread("./Datos/"+ cam + "/" + listImagenes[i])
        imgF = imgF[:, :, :, newaxis] 
        imgsFondo = np.concatenate([imgsFondo, imgF], axis = 3)
    fondo = np.median(imgsFondo, axis = 3).astype(int)
    return fondo

## Manejo de .CSV

In [22]:
def crearFichero():
    myData = [["Cam", "Date", "Ocp"]]
    myFile = open('Datos.csv', 'w', newline="")
    with myFile:
        writer = csv.writer(myFile)
        writer.writerows(myData)

In [23]:
def obtieneFichero():
    fichero = open("./Datos.csv", newline="")
    lector = csv.reader(fichero)
    rownum = 0;
    datos = [];
    for fila in lector:
        datos.append(fila)
    return datos

In [24]:
def guardarFichero(Datos):
    myFile = open('./Datos.csv', 'w', newline="")
    with myFile:
        writer = csv.writer(myFile)
        writer.writerows(Datos)

## Codigo

In [25]:
# CrearFichero
# crearFichero()

In [26]:
# Obtengo los datos del csv
#datos = obtieneFichero()

#for cont in range(0,65):
    # if cont != 63:

In [27]:
cont = 1

# Camara actual
cam = "cam" + str(cont)
       
# Listo las imagenes que hay en la carpeta
carpeta = "./Datos/" + cam
listImagenes = os.listdir(carpeta)

# Numero de imagenes necesarias para la imagen de fondo
numImagenes = 9
        
# Porcentaje de reducción de las imagenes
rd = 0.4
        
# Para cuando salgamos de una imagen NO DISPONIBLE avanzamos numImagenes para este fondo
saltaImagenes = False

# Extraigo las plantillas de la camX
plantillas = getPlantillas(cam)

# Visualizo las plantillas
# visualizaPlantillas(plantillas)

# Dimensiones de las plantillas array
size = plantillas.shape

# Redimensiono plantillas si es necesario NO PUEDO REDIMENSIONAR LAS PLANTILLAS
########################################################

# Lo encapsulo en un try-catch por si da error de lectura
try:
    # Leo la imagen de estudio
    iEstudio = plt.imread(carpeta+"/"+listImagenes[-1])
    # plt.imshow(iEstudio)
    
    # Covierto fecha y hora en un dateTime
    nombre = listImagenes[-1].split("_")
    fecha_Hora = nombre[0]+nombre[1]

    # Podemos guardarlo como dateTime
    #fecha_Hora = datetime.datetime.strptime(nombre[0]+nombre[1], '%Y%m%d%H%M')

    # Compruebo si está disponible
    disponible = camDisponible(iEstudio)
    
    if disponible:
        fondoDisponible = isFondoDisponible(cam, listImagenes)
        if fondoDisponible:           
            # Redimensiono las imagenes si es necesario NO PUEDO REDIMENSIONAR LAS IMAGENES ya que las plantillas no se pueden redimensionar
            ########################################################
            
            # Calcula el fondo
            fondo = calculaFondo(cam, listImagenes)
            # plt.imshow(fondo)
            datoInsetar = [cam, str(fecha_Hora), "cam disponible"]
        else:
            print("Fondo NO Disponible")
            datoInsetar = [cam, fecha_Hora, "Fondo NO Disponible"]
    else:
        print("Imagen No Disponible")
        datoInsetar = [cam, fecha_Hora, "Cam NO Disponible"]
    
except:
      print("Error de lectura de imagen")

# Inserto los datos nuevos
# datos.append(datoInsetar)

# Guardo el fichero con los nuevos datos
# guardarFichero(datos)

### Otros

In [69]:
# Redimensiona las plantillas pasadas por parámetros
#def redimensionaPlantilla(plantillas):   
#if size[1] > 1000:
#    x = [];
#    for i in range(0,size[0]):
#        pProvi = plantillas[i][:][:].copy()
#        pProvi.resize((576,720))
#        x.append(pProvi)
#    plantillas = np.array(x)
#        
#size = plantillas.shape
#print(size)


# tam = plantillas.shape
#a = np.array_split(plantillas,tam[2],axis=2)
#w = np.array(a[0][:][:])
#w = np.concatenate([w, w, w], axis = 2)
#for i in range(0,len(a)):
#im = Image.fromarray(w)
#plt.imshow(im)
#newsize = (720, 576) 
#im = im.resize(newsize) 
#im = np.array(im)
#print(im.shape)

### Paginas de ayuda

- https://claudiovz.github.io/scipy-lecture-notes-ES/advanced/image_processing/index.html
- https://auth0.com/blog/image-processing-in-python-with-pillow/
- https://www.youtube.com/playlist?list=PL2PZw96yQChwpPh3TR6rd9SpbPjc4Hh0M
- https://numpy.org/doc/stable/reference/generated/numpy.ndarray.resize.html
- https://numpy.org/doc/stable/reference/generated/numpy.ndarray.resize.html#numpy.ndarray.resize